# Actor-critic 알고리즘

## REINFORCE with baseline 복습

이전 장에서는 상태가치함수를 베이스라인으로 사용하여 policy gradient을 분산을 줄여주었다.

```{math}
:label: policy_gradient_baseline
\nabla_{\mathbf{\theta}} J(\mathbf{\theta}) \propto \sum_{s \in \mathcal{S}} d_{\pi_{\mathbf{\theta}}}(s) \sum_{a \in \mathcal{A}} \left( Q^{\pi_{\mathbf{\theta}}}(s,a) - V^{\pi_{\mathbf{\theta}}}(s) \right) \nabla_{\mathbf{\theta}} \pi_{\mathbf{\theta}}(a|s).
```

<br>

위에서 상태가치함수와 행동가치함수를 직접 구할 수 없기 때문에 행동가치함수를 대신하여 시뮬레이션을 통해 얻은 return $G_t$를 사용했고, 
상태가치함수는 뉴럴 네트워크 $V_\phi$를 사용하여 근사시켰다. 그래서 우리는 다음과 같은 policy gradient의 추정치를 얻었다.

```{math}
:label: reinforce_baseline_gradient
\nabla_{\theta} J(\theta) \approx \hat{g} := \frac{1}{T} \sum\limits_{t=0}^{T-1}\left( G_t - V_{\phi}(s_t)\right) \nabla_{\mathbf{\theta}} \log \pi_{\mathbf{\theta}}(a_t | s_t).
```

<br>

---


## Online actor-critic 알고리즘

Policy gradient theorem를 사용하는 방법론들 중 정책과 가치함수 모두를 뉴럴 네트워크로 모델링하는 방법론들을 Actor-critic 방법론이라고 부른다.
여기서 actor (배우)는 정책 네트워크를 의미하며 critic (비평가)는 actor의 행동을 평가하는 가치 네트워크를 말한다. 
배우-비평가 방법론이라고 부르기에는 조금 오글거리기 때문에 앞으로 actor-critic 방법론으로 부를 것이다.
하지만 REINFORCE with baseline 알고리즘은 정책과 상태가치함수를 모두 뉴럴 네트워크로 근사시키지만 actor-critic 알고리즘이라고 부르지 않는다.
그 이유에 대해서는 다양한 의견이 있지만, 
필자의 생각은 식 {eq}`policy_gradient_baseline`에서 베이스라인 부분이 아닌 $Q^{\pi_{\mathbf{\theta}}}(s,a)$를 추정하기 위하여 가치 네트워크가 사용될 때만 actor-critic 알고리즘이라고 부르는 것 같다. 
REINFORCE with baseline은 $Q^{\pi_{\mathbf{\theta}}}(s,a)$의 추정치로 return $G_t$를 사용했고, 가치 네트워크 $V_\phi$는 베이스라인으로만 사용되었기 때문에 actor-critic 방법론으로 분류되지 않는다.

<br>

그럼 $Q^{\pi_{\mathbf{\theta}}}(s,a)$를 추정하기 위해 뉴럴 네트워크를 사용하면 actor-critic 방법론이 될 것이다. 
하지만 베이스라인과 함께 사용한다고 하면 행동가치함수를 위한 네트워크 $Q_{\psi}(s, a)$와 상태가치함수를 위한 네트워크 $V_{\phi}(s)$ 2개의 네트워크를 학습시켜야 하기 때문에 학습 안정성 측면에서 안 좋을 것이다. 그래서 우리는 다음과 같은 행동가치함수의 성질을 사용할 것이다. 아래 식은 실제 상태가치함수 및 행동가치함수에 대해서 성립한다. 증명은 식 {eq}`action_bellman_equation1`에서 찾아볼 수 있다.

$$Q^{\pi}(s, a)= r(s, a) + \gamma \mathbb{E}_{s' \sim p(\cdot|s,a)}\left[ V^{\pi}(s')\right] \; \text{ for all } \; s \in \mathcal{S}, a \in \mathcal{A}.$$

<br>

따라서 가치 네트워크 $V_\phi$를 사용해서 실제 행동가치함수를 추정할 수 있다.

$$Q^{\pi}(s, a) \approx r(s, a) + \gamma \mathbb{E}_{s' \sim p(\cdot|s,a)}\left[ V_{\theta}(s') \right].$$

<br>

물론 정확한 기댓값을 구하기 어렵기 때문에 하나의 샘플로 기댓값을 대체할 것이다. 환경과 상호작용하여 얻은 $t$ 시점의 데이터 (transition) $(s_t, a_t, r_t, s_{t+1})$를 사용하여 행동가치함수를 근사시킨다. $s_{t+1}$가 전이 확률 분포 $p(\cdot|s_t, a_t)$로부터 샘플링되었기 때문에 기댓값에 대한 점 추정치로 사용될 수 있는 것이다.

```{math}
:label: actor_critic_td_target
Q^{\pi}(s_t, a_t) \approx r_t + \gamma  V_{\theta}(s_{t+1}).
```

<br>

이제 식 {eq}`reinforce_baseline_gradient`에서 행동가치함수의 추정치로 사용된 $G_t$ 대신 식 {eq}`actor_critic_td_target`을 대입해보자.

```{math}
:label: actor_critic_batch_gradient
\nabla_{\theta} J(\theta) \approx \hat{g}_{\text{batch}} := \frac{1}{T} \sum\limits_{t=0}^{T-1}\left( r_t + \gamma  V_{\theta}(s_{t+1}) - V_{\phi}(s_t)\right) \nabla_{\mathbf{\theta}} \log \pi_{\mathbf{\theta}}(a_t | s_t).
```

<br>

식 {eq}`actor_critic_batch_gradient`의 그레디언트 추정치를 사용하는 것도 하나의 actor-critic 알고리즘으로 볼 수 있다. 
하지만 식 {eq}`actor_critic_batch_gradient`에는 REINFORCE 알고리즘의 잔재가 남아 있다. 
REINFORCE의 경우 return $G_t$을 계산하기 위하여 환경의 초기 상태부터 시작하여 상호작용이 종료될 때까지 trajectory $(s_0, a_0, r_0, s_1, a_1, r_1,\ldots, s_T)$를 수집하고 네트워크 업데이트가 이뤄졌다. $G_t$를 계산할 때 $t$시점 이후로 받은 모든 보상을 더하기 때문이다. 
하지만, 식 {eq}`actor_critic_batch_gradient`은 환경이 종료될 때까지 기다릴 필요 없이 매 시점 $t$마다 환경과 상호작용하여 얻은 데이터 $(s_t, a_t, r_t, s_{t+1})$만으로 바로 네트워크 업데이트가 가능하다. 즉, 매 스탭 $t$마다 policy gradient에 대한 추정치 하나가 생긴다.

$$
\nabla_{\theta} J(\theta) \approx \hat{g}_{\text{online}} := \left( r_t + \gamma  V_{\theta}(s_{t+1}) - V_{\phi}(s_t)\right) \nabla_{\mathbf{\theta}} \log \pi_{\mathbf{\theta}}(a_t | s_t).
$$

<br>

매 스탭마다 실시간으로 구할 수 있는 그레디언트 추정치이기 때문에 $\hat{g}_{\text{online}}$라고 적어주었다. 이제 매 스탭마다 정책 네트워크를 업데이트시킬 수 있다. 
하지만 REINFORCE with base line의 경우 가치 네트워크 $V_{\phi}(s_t)$를 학습시키기 위해서 역시 return을 사용했기 때문에 환경과 상호작용이 종료되어야만 업데이트가 가능했다. 이를 보완하기 위하여 다음 실제 상태가치함수의 재귀적 성질을 이용할 것이다. 

$$V^{\pi}(s)= \mathbb{E}_{a \sim \pi(\cdot|s), s' \sim p(\cdot|s,a)} \left[ r(s, a) + \gamma  V^{\pi}(s')\right] \; \text{ for all } \; s \in \mathcal{S}.$$

<br>
 
역시나 실제 기댓값을 구할 수 없기 때문에 기댓값 대신 데이터 $(s_t, a_t, r_t, s_{t+1})$을 이용하고 다음 상태의 상태가치함수를 네트워크를 사용하여 근사시키면 다음과 같은 실제 상태가치함수에 대한 추정치를 얻을 수 있다.

$$V^{\pi}(s_t) \approx r_t + \gamma  V_{\phi}(s_{t+1}).$$

<br>

이제 $G_t$를 레이블로 사용하여 가치 네트워크 $V_{\phi}(s_{t+1})$를 학습시키는 대신 $r_t + \gamma  V_{\phi}(s_{t+1})$을 레이블 $y_t$로 사용하여 $V_{\phi}(s_{t+1})$를 학습시킨다. 구현할 때 주의할 점은 레이블로 사용되는 $y_t=r_t + \gamma  V_{\phi}(s_{t+1})$ 역시 파라미터 $\phi$를 포함하고 있다. $y_t$는 레이블로만 사용되기 때문에 실제 구현에서는 파이토치의 `detach()` 메서드를 사용하여 $y_t$를 상수 취급하고 $\phi$에 대한 그레디언트를 계산하지 않는다.